In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
summer_refractive_index = 1+0.00020409512542851616
winter_refractive_index = 1+0.00020953240736654166

import sys
import glob
from datetime import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.interpolate import interp1d
import statsmodels.api as sm

import matplotlib.pyplot as plt
from ctapipe.io import EventSource
from ctapipe.image import ImageProcessor, tailcuts_clean
from ctapipe.calib import CameraCalibrator
from ctapipe.visualization import CameraDisplay
from astropy.table import Table, vstack

#from muonpipe import usefull_func
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd



In [ ]:
1+0.00020953240736654166

In [ ]:
filename = '/Users/vdk/muons2024//simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry

start_date_2019 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397

In [ ]:
listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
listdir = listdir1 + listdir2

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    
    # Check if 'good_ring' exists and convert to bool
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_00043_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_00043_alignment

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0015_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_0015_alignment

In [ ]:
muon_files = glob.glob('/Users/vdk/Documents/DocumentsVadymMacBookWork/all_muon_fits/low_nsb_filter/*')
for i, filename in enumerate(sorted(muon_files)):
    print(i, filename)

In [ ]:
df_files = [
    pd.read_csv(muon_file, na_values=['NA', '?']).query(
        '(muon_efficiency < @muon_efficiency_cut) & '
        '(size_outside < @size_outside_cut) & '
        '(event_time >= @start_date_2024) & '
        '(event_time <= @end_date_2024)'
    )
    for muon_file in sorted(muon_files)[-1:]
]

df_good_data_real = pd.concat(df_files, ignore_index=True)
df_good_data_real

In [ ]:
ring_completeness_cut = 0
ring_containment_cut = 0
min_radius_cut = 0.5
max_radius_cut = 1.5
#ring_center_distance_cut = 0.6
#impact_distance_cut = 6

# ring_completeness_cut = 0.4
# ring_containment_cut = 0
# min_radius_cut = 0.6
# max_radius_cut = 1.4
ring_center_distance_cut = 2
impact_distance_cut = 15


In [ ]:
def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

df_sim_00043_cut = df_sim_00043_alignment[
    (df_sim_00043_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_00043_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_00043_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_00043_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_00043_alignment['ring_center_x'].abs() < ring_center_distance_cut) &
    (df_sim_00043_alignment['ring_center_y'].abs() < ring_center_distance_cut) &
    (df_sim_00043_alignment['impact_parameter'] < impact_distance_cut)
]
print(f"Events survived for df_sim_00043_cut: {len(df_sim_00043_cut)} ({calculate_survival_percentage(df_sim_00043_alignment, df_sim_00043_cut):.2f}%)")

df_sim_0015_cut = df_sim_0015_alignment[
    (df_sim_0015_alignment['ring_completeness'] > ring_completeness_cut) &
    (df_sim_0015_alignment['ring_containment'] > ring_containment_cut) &
    (df_sim_0015_alignment['ring_radius'] > min_radius_cut) &
    (df_sim_0015_alignment['ring_radius'] < max_radius_cut) &
    (df_sim_0015_alignment['ring_center_x'].abs() < ring_center_distance_cut) &
    (df_sim_0015_alignment['ring_center_y'].abs() < ring_center_distance_cut) &
    (df_sim_0015_alignment['impact_parameter'] < impact_distance_cut)
]
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_alignment, df_sim_0015_cut):.2f}%)")

df_data_cut = df_good_data_real[
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &
    (df_good_data_real['ring_radius'] < max_radius_cut) &
    (df_good_data_real['ring_center_x'].abs() < ring_center_distance_cut) &
    (df_good_data_real['ring_center_y'].abs() < ring_center_distance_cut) &
    (df_good_data_real['impact_parameter'] < impact_distance_cut)
]
print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%)")

In [ ]:
df_data_cut.columns

In [ ]:
bin_number = 70
plt.figure(figsize=(12,9))
bins = np.linspace(0, 3500, bin_number)
plt.hist(df_sim_0015_cut['ring_size'], alpha=0.9, bins=bins, density=True, label='simulations', histtype='step', lw=3, color='orange')
plt.hist(df_data_cut['ring_size'], alpha=1, bins=bins, density=True, label='data', histtype='step', lw=3, color='k')
#plt.hist(df_data_cut['ring_size'], alpha=1, bins=bins, density=True, label='data', histtype='step', lw=3, color='k')

#plt.hist(df_sim_0015_alignment['ring_size'],alpha = 0.3, bins = 50, density=True, label='simulations')

mean_data = df_data_cut['ring_size'].mean()
std_data = df_data_cut['ring_size'].std()

mean_0015 = df_sim_0015_cut['ring_size'].mean()
std_0015 = df_sim_0015_cut['ring_size'].std()

# Print mean and std on the figure
plt.text(2800, 0.002, f"Mean: {mean_data:.3f}, Std: {std_data:.3f}", fontsize=18, color='k')
plt.text(2800, 0.0015, f"Mean: {mean_0015:.3f}, Std: {std_0015:.3f}", fontsize=18, color='orange')

plt.legend(prop={'size': 20})
plt.xlabel('Ring size', fontsize = 20)
plt.ylabel('Normalized counts', fontsize = 20)
plt.xlim(0,4000)
plt.grid(alpha = 0.5)
plt.show()

In [ ]:
min(df_data_cut['size_outside'])

In [ ]:
bin_number = 70
plt.figure(figsize=(12,9))
bins = np.linspace(0, 100, bin_number)
plt.hist(df_sim_0015_cut['size_outside'][df_sim_0015_cut['size_outside'] > 1e-10], alpha=0.9, bins=bins, density=True, label='simulations', histtype='step', lw=3, color='orange', log=True)
plt.hist(df_data_cut['size_outside'][df_data_cut['size_outside'] > 1e-10], alpha=1, bins=bins, density=True, label='data', histtype='step', lw=3, color='k', log=True)
#plt.hist(df_data_cut['ring_size'], alpha=1, bins=bins, density=True, label='data', histtype='step', lw=3, color='k')

#plt.hist(df_sim_0015_alignment['ring_size'],alpha = 0.3, bins = 50, density=True, label='simulations')

#mean_data = df_data_cut['ring_size'].mean()
#std_data = df_data_cut['ring_size'].std()

# mean_0015 = df_sim_0015_cut['ring_size'].mean()
# std_0015 = df_sim_0015_cut['ring_size'].std()

# # Print mean and std on the figure
# plt.text(2800, 0.002, f"Mean: {mean_data:.3f}, Std: {std_data:.3f}", fontsize=18, color='k')
# plt.text(2800, 0.0015, f"Mean: {mean_0015:.3f}, Std: {std_0015:.3f}", fontsize=18, color='orange')

plt.legend(prop={'size': 20})
plt.xlabel('Size outside', fontsize = 20)
plt.ylabel('Normalized counts', fontsize = 20)
plt.xlim(0,100)
plt.grid(alpha = 0.5)
plt.show()

In [ ]:
year_value = 2024
data_mc = df_sim_0015_alignment
data_run = df_good_data_real
min_radius_cut = 0.9
max_radius_cut = 1.2

bins = np.linspace(min_radius_cut, max_radius_cut, 10)

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_mc['ring_size'] = binned_mc['ring_size']
binned_mc['constant'] = 1
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()

# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run['constant'] = 1
model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()
y_values_run = model_run.predict(x_values_with_const)
prediction_run = model_run.get_prediction(x_values_with_const)
conf_int_run = prediction_run.conf_int()


# Plotting
plt.figure(figsize=(12, 12))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation optical efficiency tuned to 2024 year ', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression line')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label=f'Data for {year_value} year', s=50)

plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression line')
plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color='orange', alpha=0.2)


# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.ylim(1800,3000)
#plt.savefig(f'/Users/vdk/muons2024/images/ring_size_check/full_sample_2024NSBtune_size_vs_radius_{year_value}year_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
def cherenkov_angle(muon_energy_eV, refractive_index=1.000217):
    """
    Calculate the Cherenkov angle for a given muon energy and refractive index.

    Parameters:
    - muon_energy_eV (float): The energy of the muon in electron volts (eV).
    - refractive_index (float, optional): The refractive index of the medium. Default is 1.000293.

    Returns:
    - theta_C_degrees (float): The Cherenkov angle in degrees.

    Raises:
    - None

    """
    # Constants
    c = 299792458  # speed of light in m/s
    muon_rest_mass_MeV = 105.66  # rest mass of muon in MeV
    eV_to_MeV = 1e-6  # conversion factor from eV to MeV
    
    # Convert muon energy from eV to MeV
    muon_energy_MeV = muon_energy_eV * eV_to_MeV
    
    # Check if the energy is sufficient for Cherenkov radiation
    if muon_energy_MeV <= muon_rest_mass_MeV:
        return "Energy too low for Cherenkov radiation."
    
    # Calculate the momentum p
    p = math.sqrt((muon_energy_MeV)**2 - (muon_rest_mass_MeV)**2)
    
    # Calculate beta
    beta = p / muon_energy_MeV
    
    # Check if Cherenkov condition is met
    if beta * refractive_index <= 1:
        return "Muon velocity too low for Cherenkov radiation in this medium."
    
    # Calculate Cherenkov angle
    cos_theta_C = 1 / (beta * refractive_index)
    theta_C = math.acos(cos_theta_C)
    
    # Convert angle from radians to degrees
    theta_C_degrees = math.degrees(theta_C)
    
    return theta_C_degrees


def plot_cherenkov_angle_distribution(min_energy_eV, max_energy_eV, num_points, refractive_index=1.000217, plot_option=False):
    """
    Plots the Cherenkov angle distribution as a function of muon energy.

    Parameters:
    - min_energy_eV (float): The minimum energy in electron volts (eV) for the muon.
    - max_energy_eV (float): The maximum energy in electron volts (eV) for the muon.
    - num_points (int): The number of points to generate between the minimum and maximum energy.
    - refractive_index (float, optional): The refractive index of the medium. Default is 1.000293.
    - plot_option (bool, optional): Whether to plot the distribution. Default is False.

    Returns:
    - energies_GeV (numpy.ndarray): An array of muon energies in gigaelectron volts (GeV).
    - angles (list): A list of Cherenkov angles in degrees corresponding to the muon energies.

    Example usage:
    >>> min_energy_eV = 1e6
    >>> max_energy_eV = 1e9
    >>> num_points = 100
    >>> refractive_index = 1.000293
    >>> plot_option = True
    >>> energies_GeV, angles = plot_cherenkov_angle_distribution(min_energy_eV, max_energy_eV, num_points, refractive_index, plot_option)
    """

    energies_eV = np.logspace(np.log10(min_energy_eV), np.log10(max_energy_eV), num_points)
    energies_GeV = energies_eV * 1e-9  # Convert eV to GeV
    angles = []

    for energy in energies_eV:
        angle = cherenkov_angle(energy, refractive_index)
        if angle is not None:
            angles.append(angle)
        else:
            angles.append(0)  # Append 0 for energies not producing Cherenkov radiation

    if plot_option:
        plt.plot(energies_GeV, angles, label='Cherenkov Angle')
        plt.xscale('log')
        plt.xlabel('Muon Energy (GeV)')
        plt.ylabel('Cherenkov Angle (degrees)')
        plt.title('Cherenkov Angle Distribution as a Function of Muon Energy')
        plt.legend()
        plt.show()

    return energies_GeV, angles

In [ ]:
cherenkov_angle(muon_energy_eV=1e12)

In [ ]:
min_energy_eV = 6e9  # 1 GeV
max_energy_eV = 1e12  # 1 TeV
num_points = 1000
test_refractive_index = 1 + 2.29611e-04

km2_winter_index = 1 + 0.00023252249318383988
km2_inter_index = 1 + 0.00022933454584772583
km2_summer_index = 1 + 0.00022516304754782723

energies, summer_angles = plot_cherenkov_angle_distribution(min_energy_eV, max_energy_eV, num_points,refractive_index=km2_summer_index)

In [ ]:
max(summer_angles)

In [ ]:
energies, winter_angles = plot_cherenkov_angle_distribution(min_energy_eV, max_energy_eV, num_points,refractive_index=km2_winter_index)
max(winter_angles)

In [ ]:
plt.figure(figsize=(12,9))
print(f"Maximum angle value for the summer = {round(max(summer_angles),3)} degrees")
#print(f"Maximum angle value for the winter = {round(max(winter_angles),3)} degrees")
plt.plot(energies, summer_angles, label='Cherenkov Angle for Summer')
#plt.plot(energies, winter_angles, label='Cherenkov Angle for  Winter')
plt.xscale('log')
plt.xlabel('Muon Energy (GeV)')
plt.ylabel('Cherenkov Angle (degrees)')
#plt.title('Cherenkov Angle Distribution as a Function of Muon Energy and refractive index')
plt.legend()
#plt.savefig('/Users/vdk/muons2024/images/AA_paper/cherenkov_angle_distribution.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
df_data_cut.columns


In [ ]:
name_of_columns = ['ring_size', 'size_outside', 'ring_center_x', 'ring_center_y', 'ring_radius', 'ring_width', 'num_pixels_in_ring', 'mean_pixel_charge_around_ring', 'ring_completeness', 'ring_containment']
def plot_columns(df_data, df_sim, columns):
    fig, axes = plt.subplots(4, 2, figsize=(15, 20))
    axes = axes.flatten()

    for i, col in enumerate(columns):
        num_bins = min(50, max(10, int(np.sqrt(len(df_data[col].dropna())))))
        axes[i].hist(df_data[col].dropna(), bins=num_bins, alpha=0.7, color='k', density=True, label='data')
        axes[i].hist(df_sim[col].dropna(), bins=num_bins, alpha=0.7, color='orange', density=True, label='sim')
        #axes[i].hist(df_data[col].dropna(), bins=50, alpha=0.7, color='k', density=True, label = 'data')
        #axes[i].hist(df_sim[col].dropna(), bins=50, alpha=0.7, color='orange', density=True, label = 'sim')
        axes[i].set_title(f'Histogram of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')
        axes[i].legend()

    plt.tight_layout()
    plt.show()

plot_columns(df_data_cut, df_sim_0015_cut, name_of_columns)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

def plot_columns(df_data, df_sim, columns):
    fig, axes = plt.subplots(5, 2, figsize=(15, 20))
    axes = axes.flatten()

    for i, col in enumerate(columns):
        data_values = df_data[col].dropna()
        sim_values = df_sim[col].dropna()

        combined_values = pd.concat([data_values, sim_values])

        min_value = combined_values.min()
        max_value = combined_values.max()

        total_events = len(combined_values)
        num_bins = int(np.sqrt(total_events))  

        num_bins = max(10, min(num_bins, 100)) 

        bins = np.linspace(min_value, max_value, num_bins + 1)

        axes[i].hist(data_values, bins=bins, alpha=0.7, color='k', density=True, label='Data')
        axes[i].hist(sim_values, bins=bins, alpha=0.7, color='orange', density=True, label='Simulation')

        axes[i].set_title(f'Histogram of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Normalized Frequency')
        axes[i].legend()

    plt.tight_layout()
    plt.show()
    
plot_columns(df_data_cut, df_sim_0015_cut, name_of_columns)

In [ ]:
plot_columns(df_data_cut[(abs(df_data_cut['ring_center_x']) < 0.5) & (abs(df_data_cut['ring_center_y']) < 0.5)], df_sim_0015_cut[(abs(df_sim_0015_cut['ring_center_x']) < 0.5) & (abs(df_sim_0015_cut['ring_center_y']) < 0.5)], name_of_columns)

In [ ]:
plot_columns(df_data_cut[
                (abs(df_data_cut['ring_center_x']) < 0.5) & 
                (abs(df_data_cut['ring_center_y']) < 0.5) &
               (df_data_cut['ring_completeness'] > 0.99) &
               (df_data_cut['ring_radius'] < 1.15)
    ],
            df_sim_0015_cut[
                (abs(df_sim_0015_cut['ring_center_x']) < 0.5) & 
                (abs(df_sim_0015_cut['ring_center_y']) < 0.5) &
                (df_sim_0015_cut['ring_completeness'] > 0.99) &
                (df_sim_0015_cut['ring_radius'] < 1.16)
    ], 
    name_of_columns)

In [ ]:
plt.hist(df_data_cut['ring_radius'], bins = 20, density=True)
plt.xlim(0.5, 1.5)

In [ ]:
plt.hist(df_sim_0015_cut['mc_energy'], bins = 20, density=True, log=True)

In [ ]:
x is blank.x

In [ ]:
blank.y = 10000

In [ ]:
y = blank.y

In [ ]:
class Hs:
    """ """